# 마우스 얼굴 표정 모델 학습

이 노트북은 MacBook에서 CPU를 사용하여 모델을 학습합니다.

## 예상 시간
- 1 epoch: 약 40-50분
- 10 epochs: 약 7-8시간 (밤새 실행 권장)

## 1단계: 환경 설정 및 확인

In [ ]:
import os
import sys

# 프로젝트 디렉토리 설정
project_dir = '/Users/minakang/Desktop/mouse-facial-expressions-2023-main'
os.chdir(project_dir)

print(f"현재 디렉토리: {os.getcwd()}")
print(f"Python 버전: {sys.version}")

In [ ]:
# 필수 라이브러리 확인
import torch
import lightning
import mlflow
import pandas as pd
import numpy as np

print(f"PyTorch 버전: {torch.__version__}")
print(f"Lightning 버전: {lightning.__version__}")
print(f"MLflow 버전: {mlflow.__version__}")
print(f"\nCUDA 사용 가능: {torch.cuda.is_available()}")
print(f"MPS 사용 가능: {torch.backends.mps.is_available()}")
print(f"\n⚠️ 이 노트북은 CPU로 학습합니다 (안정성을 위해)")

## 2단계: 데이터 확인

In [ ]:
# 데이터 디렉토리 확인
data_dir = os.path.join(project_dir, 'data')
frames_dir = os.path.join(data_dir, 'local_frames')
processed_dir = os.path.join(data_dir, 'processed', 'task-1.1')

print(f"데이터 디렉토리 존재: {os.path.exists(data_dir)}")
print(f"프레임 디렉토리 존재: {os.path.exists(frames_dir)}")
print(f"처리된 데이터 디렉토리 존재: {os.path.exists(processed_dir)}")

if os.path.exists(frames_dir):
    frame_count = sum(1 for _ in os.walk(frames_dir) for f in os.listdir(_[0]) if f.endswith('.png'))
    print(f"\n총 프레임 수: {frame_count}")

if os.path.exists(processed_dir):
    pkl_files = [f for f in os.listdir(processed_dir) if f.endswith('.pkl')]
    print(f"\n처리된 데이터 파일:")
    for f in pkl_files:
        print(f"  - {f}")

## 3단계: 환경 변수 설정

In [ ]:
# .env 파일 생성/업데이트
env_content = f"""MFE_RAW_VIDEO_FOLDER={project_dir}/data/raw
MFE_RAW_CSV_FOLDER={project_dir}/data/raw
MFE_VERSION=20230627
MFE_PROCESSED_VIDEO_FOLDER={project_dir}/data/processed
MFE_EXTRACTED_FRAMES_FOLDER={project_dir}/data/local_frames
MFE_TASKS={project_dir}/data/processed
"""

with open('.env', 'w') as f:
    f.write(env_content)

print("✅ .env 파일 생성 완료")
print("\n내용:")
print(env_content)

## 4단계: 데이터셋 준비 (필요한 경우만 실행)

⚠️ 이미 `data/processed/task-1.1/` 디렉토리에 `.pkl` 파일들이 있다면 이 단계를 건너뛰세요.

In [ ]:
# 데이터셋이 없는 경우에만 실행
if not os.path.exists(processed_dir) or len([f for f in os.listdir(processed_dir) if f.endswith('.pkl')]) == 0:
    print("데이터셋을 생성합니다...")
    !python mouse_facial_expressions/data/make_datasets.py task1 --version "1.1" --frameset_size 1
else:
    print("✅ 데이터셋이 이미 존재합니다. 이 단계를 건너뜁니다.")

## 5단계: 학습 시작

### 옵션 A: 테스트 실행 (1 epoch, 약 40-50분)

In [ ]:
# 1 epoch 테스트 실행
!python mouse_facial_expressions/models/train_task1_baseline_model.py \
    --epochs 1 \
    --dataset_version "1.1"

### 옵션 B: 전체 학습 (10 epochs, 약 7-8시간)

⚠️ 이 셀은 오래 걸립니다. 밤새 실행하는 것을 권장합니다.

In [ ]:
# 전체 학습 실행 (10 epochs)
!python mouse_facial_expressions/models/train_task1_baseline_model.py \
    --epochs 10 \
    --dataset_version "1.1"

## 6단계: 학습 모니터링

학습이 진행되는 동안 다른 셀에서 진행 상황을 확인할 수 있습니다.

In [ ]:
# 체크포인트 확인
checkpoint_dir = os.path.join(project_dir, 'models', 'checkpoints')
if os.path.exists(checkpoint_dir):
    checkpoints = [f for f in os.listdir(checkpoint_dir) if f.endswith('.ckpt')]
    if checkpoints:
        print(f"저장된 체크포인트 ({len(checkpoints)}개):")
        for ckpt in sorted(checkpoints):
            size = os.path.getsize(os.path.join(checkpoint_dir, ckpt)) / (1024*1024)
            print(f"  - {ckpt} ({size:.1f} MB)")
    else:
        print("아직 체크포인트가 없습니다.")
else:
    print("체크포인트 디렉토리가 아직 생성되지 않았습니다.")

In [ ]:
# MLflow 실험 결과 확인
import mlflow

mlflow.set_tracking_uri("file://" + os.path.join(project_dir, "mlruns"))

# 최근 실험 가져오기
client = mlflow.tracking.MlflowClient()
experiments = client.search_experiments()

if experiments:
    print("MLflow 실험:")
    for exp in experiments:
        print(f"\n실험 ID: {exp.experiment_id}")
        print(f"이름: {exp.name}")
        
        # 실험의 최근 run 가져오기
        runs = client.search_runs(exp.experiment_id, max_results=5)
        if runs:
            print(f"\n최근 실행 ({len(runs)}개):")
            for run in runs:
                print(f"  - Run ID: {run.info.run_id[:8]}...")
                print(f"    상태: {run.info.status}")
                if run.data.metrics:
                    print(f"    메트릭: {dict(list(run.data.metrics.items())[:3])}")
else:
    print("아직 MLflow 실험이 없습니다.")

## 7단계: 결과 확인

In [ ]:
# 최고 성능 모델 찾기
import re

checkpoint_dir = os.path.join(project_dir, 'models', 'checkpoints')
if os.path.exists(checkpoint_dir):
    checkpoints = [f for f in os.listdir(checkpoint_dir) if f.endswith('.ckpt')]
    
    best_acc = 0
    best_ckpt = None
    
    for ckpt in checkpoints:
        # 파일명에서 val_acc 추출
        match = re.search(r'val_acc=([0-9.]+)', ckpt)
        if match:
            acc = float(match.group(1))
            if acc > best_acc:
                best_acc = acc
                best_ckpt = ckpt
    
    if best_ckpt:
        print(f"✅ 최고 성능 모델:")
        print(f"   파일: {best_ckpt}")
        print(f"   검증 정확도: {best_acc:.4f}")
    else:
        print("아직 검증 정확도가 기록된 체크포인트가 없습니다.")
else:
    print("체크포인트 디렉토리가 없습니다.")

## 8단계: 학습 로그 확인 (선택사항)

In [ ]:
# 학습 로그 파일 확인
log_file = os.path.join(project_dir, 'training_log.txt')
if os.path.exists(log_file):
    with open(log_file, 'r') as f:
        lines = f.readlines()
        print(f"학습 로그 (마지막 20줄):")
        print(''.join(lines[-20:]))
else:
    print("학습 로그 파일이 없습니다.")

## 유용한 팁

### 학습 중단하기
- Jupyter 메뉴: Kernel → Interrupt
- 또는 셀 실행 중 ■ 버튼 클릭

### 학습 재개하기
체크포인트에서 학습을 재개하려면:
```python
!python mouse_facial_expressions/models/train_task1_baseline_model.py \
    --epochs 10 \
    --dataset_version "1.1" \
    --resume_from_checkpoint "models/checkpoints/your_checkpoint.ckpt"
```

### MLflow UI 실행
터미널에서:
```bash
cd /Users/minakang/Desktop/mouse-facial-expressions-2023-main
mlflow ui
```
그런 다음 브라우저에서 http://localhost:5000 접속